# 3 Coloring Zero Knowledge Proof
The following shows an interactive proof for Graph 3 Coloring. Recall that a $k$-col for a graph $G = (V, E)$ is a function $\pi: V \to \mathbb{N}_k$ such that $(u, v) \in E \implies \phi(u) \neq \phi(v)$. So a mapping to the set $\{1, \dots, k \}$ such that neighbouring vertexes are mapped to different classes.  
We make use of commitments, which are a way to ensure that the prover chooses some values that it cannot then arbitrarly change without the verifier knowing. In particular, we use Python's pseudorandom generator for this aim (Note that this is not a particolarly strong commitment scheme, but for illustration purposes it should be fine).
The protocol works as following:
1. The prover selects a random permutation $\sigma$ and sets $\phi \equiv \sigma \circ \pi$
2. It then select random strings $r_i$ and computes the commitments $c_i \equiv C_{r_i}(\phi(i))$. The provers sends $c_1, \dots\, c_n$ to the verifier
3. The verifier selects at random an edge $(u, v)$
4. The prover sends the reveal of the commitments of $c_u, c_v$, namely $(r_u, \phi(u)), (r_v, \phi(v))$, to the verifier
5. Finally, the verifier checks that $\phi(u) \neq \phi(v)$ and that $C_{r_u}(\phi(u)) = c_u$ and $C_{r_v}(\phi(v))$. If it holds then it accepts

Let us start showing the protocol. In particular, first we define the parameters and the routines that we will be using. We will generate some random tripartite graphs, which almost by definition are 3 colorable. 

In [40]:
V = 12
E = 40

def generate_with_three_col(V, E):
    # We generate a random three partite
    if V % 3 != 0:
        return
    vertexes = list(range(1, V + 1))
    chunk_size = V // 3
    # First divide in three sets
    partitions = [vertexes[:chunk_size],vertexes[chunk_size:2*chunk_size],vertexes[2*chunk_size:]]
    edges = set()
    while E > 0:
        # Chose two distinct classes
        first_class = randrange(0, 3)
        second_class = choice(list(set([0, 1, 2]).difference([first_class])))
        # Chose two distinct nodes and make an edge
        first_node, second_node = randrange(0, chunk_size), randrange(0, chunk_size)
        edge = (partitions[first_class][first_node], partitions[second_class][second_node])
        if edge in edges:
            continue
        edges.add(edge)
        E = E - 1
    
    return Graph(list(edges)), tripartite_col(V)

def tripartite_col(V):
    chunk = V // 3
    return lambda x: 1 if x in range(1, chunk + 1) else 2 if x in range(chunk + 1, 2 * chunk + 1) else 3

def commit(color, random, N):
    # We use a fixed seed to reproduce
    set_random_seed(random)
    r = current_randstate().python_random(seed=random)
    return (r.getrandbits(3 * N), ((random ^ random) % 2) ^ color)
    
def compose(function, perm):
    return lambda i: perm(function(i))



We generate the graph, and find the coloring that is taken as private to the prover. Then we select a random $\sigma$ and permute the coloring $\pi$ with it

In [ ]:
G, pi = generate_with_three_col(V, E)
sigma = Permutation(Permutations(3).random_element())
phi = compose(pi, sigma)

random_strings = [getrandbits(V) for _ in G.vertices()]
commitments = map(lambda (a, b): commit(a, b, V), zip([phi(i) for i in G.vertices()], random_strings))

G.show()

Now the verifier selects a random edge as a challenge

In [42]:
u, v = G.random_edge(labels = False)
(u, v)

(3, 6)

The prover sends to the verifier the values that it used to commit the selected edges. (The factor of -1 is due to the fact that arrays in python are 0 indexed)

In [43]:
r_u = random_strings[u - 1]
color_u = phi(u)
r_v = random_strings[v - 1]
color_v = phi(v)

(r_u, color_u), (r_v, color_v)

((2273L, 1), (198L, 2))

Finally, the verifier checks that the values are consistent for 3 coloring, and that the commitments are valid. If so it accepts

In [44]:
coloring_valid = color_u != color_v
valid_start_commitment = commitments[u - 1] == commit(color_u, r_u, V)
valid_end_commitment = commitments[v - 1] == commit(color_v, r_v, V)
coloring_valid and valid_start_commitment and valid_end_commitment

True

# Experiment
There are essentially three cases that are important for us. First of all, if a graph is 3 colorable and we have such a coloring, then the prover will be able to easily convince the verifier with probability one. 
Secondly, in case the graphs are selected randomly, we expect the prover to be able to fool the verifier at least $2/3$ of the times (as it can select vertices colors at random). In fact, we show that a simulator can simulate this and succeed that percentage of the times. 
Thirdly, we can have the prover have an almost perfect three coloring of the graph (e.g. one in which a single edge fails the criteria). In that case, it will fool the verifier with probability $1 - \frac{1}{|E|}$.

First of all, let us model the honest case


In [45]:
N = 500

class HonestVerifier:
    def __init__(self, G):
        self.G = G
        
    def challenge(self, commitments):
        self.commitments = commitments
        self.u, self.v = self.G.random_edge(labels = False)
        return (self.u, self.v)
    
    def verify(self, r_u, color_u, r_v, color_v):
        coloring_valid = color_u != color_v
        valid_start_commitment = self.commitments[self.u - 1] == commit(color_u, r_u, V)
        valid_end_commitment = self.commitments[self.v - 1] == commit(color_v, r_v, V)
        return coloring_valid and valid_start_commitment and valid_end_commitment
        
    
def procedure_honest_prover(G, pi, ver):
    V = len(G.vertices())
    
    # First we permute the original coloring
    sigma = Permutation(Permutations(3).random_element())
    phi = compose(pi, sigma)

    # Compute the commitments
    random_strings = [getrandbits(V) for _ in G.vertices()]
    commitments = map(lambda (a, b): commit(a, b, V), zip([phi(i) for i in G.vertices()], random_strings))
    
    # Verifier init and challenge
    verifier = ver(G)
    u, v = verifier.challenge(commitments)
    
    # Get the values used for committing the values
    r_u = random_strings[u - 1]
    color_u = phi(u)
    r_v = random_strings[v - 1]
    color_v = phi(v)
    
    # Check if the verifier accepts
    return verifier.verify(r_u, color_u, r_v, color_v)

def experiment_honest(V, E, N):
    success_count = 0
    for i in range(0, N):
        G, pi = generate_with_three_col(V, E)      
        success_count += procedure_honest_prover(G, pi, HonestVerifier)
    return success_count / N



As we can see, the honest experiment will succeed with probability 1

In [46]:
experiment_honest(V, E, N)

1

Now, we model a dishonest prover, that uses an almost perfect coloring to try to fool the verifier. Note how we also allow repetition in this model, which we will use to reduce the soundness bound exponentially

In [47]:
def procedure_dishonest_prover(G, ver):
    V = len(G.vertices())
    
    # We cheat here, using the original coloring, and hoping the verifier does not find out
    pi = tripartite_col(V)
    
    # First we permute the original coloring
    sigma = Permutation(Permutations(3).random_element())
    phi = compose(pi, sigma)

    # Compute the commitments
    random_strings = [getrandbits(V) for _ in G.vertices()]
    
    commitments = map(lambda (a, b): commit(a, b, V), zip([phi(i) for i in G.vertices()], random_strings))
    
    # Verifier init and challenge
    verifier = ver(G)
    u, v = verifier.challenge(commitments)
    
    # Get the values used for committing the values
    r_u = random_strings[u - 1]
    color_u = phi(u)
    r_v = random_strings[v - 1]
    color_v = phi(v)
    
    # Check if the verifier accepts
    return verifier.verify(r_u, color_u, r_v, color_v)

def almost_3col(V, E):
    G, _ = generate_with_three_col(V, E)
    G.add_edge(1, 2)
    return G

def experiment_dishonest(V, E, verifier, procedure, graph_gen, works, N, K):
    success_count = 0
    for i in range(0, N):
        # We generate a graph, and add a single edge in a partition
        G = graph_gen(V, E)
        failure = False
        for j in range(0, K):
            if not works(procedure(G, verifier)):
                failure = True
                break
        success_count += not failure
    return success_count / N

Now, here the success probability of this dishonest prover. We can see check what the difference between that and the predicted fooling gap, namely $1 - \frac{1}{|E|}$ is

In [48]:
success_prob = experiment_dishonest(V, E, HonestVerifier, procedure_dishonest_prover, almost_3col, lambda x: x, N, 1)
expected = 1 - 1/E
success_prob, abs(success_prob - expected)

(487/500, 1/1000)

Finally, we show a simulator that succeeds with probability 2/3

In [49]:
def simulator(G, ver):
    V = len(G.vertices())
    
    # Fake coloring (select each vertex at random)
    func = [choice([1,2,3]) for _ in G.vertices()]
    
    # Compute the commitments
    random_strings = [getrandbits(V) for _ in G.vertices()]
    
    commitments = map(lambda (a, b): commit(a, b, V), zip(func, random_strings))
    
    # Verifier init and challenge
    verifier = ver(G)
    u, v = verifier.challenge(commitments)
    if not G.has_edge(u,v):
        u, v = G.edges()[0]
    
    if func[u-1] == func[v-1]:
        return (False, None)
    
    r_u = random_strings[u - 1]
    color_u = func[u-1]
    r_v = random_strings[v - 1]
    color_v = func[v-1]
    
    return (G, commitments, (r_u, color_u, r_v, color_v))

We actually use the non 3 colorable graphs that we used in the dishonest verifier case, as this simulator does not take advantage of that in any way. As you can see, the probability hovers around 2/3

In [50]:
experiment_dishonest(V, E, HonestVerifier, simulator, almost_3col, lambda x: x[0], N, 1)

77/125

Finally, we show that the soundness gap can be reduced quickly using repetition. Exponentially quickly in fact. Note that the exponential is comparable with the $(1 - \frac{1}{|E|})^k$. (Also, this might take a while to run)

In [ ]:
list_plot(
    [experiment_dishonest(V, E, HonestVerifier, procedure_dishonest_prover, almost_3col, lambda x: x, N, k)
     for k in range(0, 8)]
)